In [ ]:
#Set up the directory
new_dir = "/home/student/data_5000_project/programs/R-CNN/"
%cd $new_dir
!pwd

In [ ]:
!python -V

Run
!sudo git clone https://github.com/matterport/Mask_RCNN.git

Data is in 
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum

In [ ]:
!pwd


In [ ]:
#if not /home/student/data_5000_project/programs/R-CNN/Mask_RCNN
%cd Mask_RCNN

In [ ]:
!python setup.py install

In [ ]:
!pip install scikit-image

In [ ]:
!pip show mask-rcnn
!pwd

In [ ]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset

# class that defines and loads the opus_incertum dataset
class ObjectDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define one class
        self.add_class("dataset", 1, "opus_incertum")
        # define data locations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        # find all images
        for filename in listdir(images_dir):
            # extract image id
            image_id = filename[:-4]
            
            # skip all images after 400 if we are building the train set
            if is_train and int(image_id) >= 400:
                continue
            # skip all images before 400 if we are building the test/val set
            if not is_train and int(image_id) < 400:
                continue
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            # add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    # count bounding boxes from an annotation file
    def count_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        box_count=0
        for box in root.findall('.//bndbox'):
            box_count=box_count+1
        return box_count

    
    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        path = info['annotation']
        # load XML
        boxes, w, h = self.extract_boxes(path)
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('opus_incertum'))
        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

# train set
train_set = ObjectDataset()
# opus_incertum - change
train_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# test/val set
test_set = ObjectDataset()
# opus_incertum - change
test_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# Count Boxes for Confusion Matrix

In [ ]:
import inspect
#print(inspect.getmembers(train_set))
#print(train_set)
print("Train")
print('Train images: %d' % len(train_set.image_ids))
boxes_total = 0
for i in train_set.image_info:
    #print(i['annotation'])
    boxes_count = train_set.count_boxes(i['annotation'])
    boxes_total = boxes_total + boxes_count
    #print(boxes_count)
print('Train boxes: %d' % boxes_total)

print("Test")
print('Test images: %d' % len(test_set.image_ids))
boxes_total = 0
for i in test_set.image_info:
    #print(i['annotation'])
    boxes_count = train_set.count_boxes(i['annotation'])
    boxes_total = boxes_total + boxes_count
    #print(boxes_count)
print('Test boxes: %d' % boxes_total)

In [ ]:
# plot one photograph and mask
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from matplotlib import pyplot



# train set
train_set = ObjectDataset()
#opus_incertum
train_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=True)
train_set.prepare()
# load an image
image_id = 8
image = train_set.load_image(image_id)
print(image.shape)
# load image mask
mask, class_ids = train_set.load_mask(image_id)
print(mask.shape)
# plot image
pyplot.imshow(image)
# plot mask
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)
pyplot.show()

In [ ]:
# display image with masks and bounding boxes
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset

from mrcnn.utils import extract_bboxes

# train set
train_set = ObjectDataset()
train_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=True)
train_set.prepare()
# define image id
image_id = 6
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

In [ ]:
print(image)



In [ ]:
# fit a mask rcnn on the opus_incertum dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN



# define a configuration for the model
class ObjectConfig(Config):
    # define the name of the configuration
    NAME = "opus_incertum_cfg"
    # number of classes (background + opus_incertum)
    NUM_CLASSES = 1 + 1
    # number of training steps per epoch
    STEPS_PER_EPOCH = 160
    VALIDATION_STEPS = 40

# prepare train set
train_set = ObjectDataset()
train_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = ObjectDataset()
test_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = ObjectConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=20, layers='heads')

#Thanks to Renu Khandelwal https://towardsdatascience.com/object-detection-using-mask-r-cnn-on-a-custom-dataset-4f79ab692f6d 
history = model.keras_model.history.history 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[8,6])
plt.plot(history['val_loss'],'r',linewidth=3.0)
#plt.plot(history.history['val_acc'],'b',linewidth=3.0)
#plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
#plt.ylabel('Accuracy',fontsize=16)
plt.title('Still working on this',fontsize=16)

In [ ]:
print(history)

In [ ]:
# evaluate the mask rcnn model on the opus_incertum dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image



# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "opus_incertum_cfg"
	# number of classes (background + opus_incertum)
	NUM_CLASSES = 1 + 1
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# load the train dataset
train_set = ObjectDataset()
train_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = ObjectDataset()
test_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# create config
cfg = PredictionConfig()
# define the model
#cfg_name = 'cfg20200208T0930'
#cfg_name = 'cfg20200301T1702'
#cfg_name = ''
cfg_name = 'cfg20200307T0901'
print(cfg_name)
model = MaskRCNN(mode='inference', model_dir='/opus_incertum_'+cfg_name, config=cfg)
# load model weights
model.load_weights('/home/student/data_5000_project/programs/R-CNN/Mask_RCNN/opus_incertum_'+cfg_name+'/mask_rcnn_opus_incertum_cfg_0005.h5', by_name=True)

# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

In [ ]:
# detect opus_incertum in photos 
import os
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset



# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "opus_incertum_cfg"
    # number of classes (background + opus_incertum)
    NUM_CLASSES = 1 + 1
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, image_prediction_folder, n_images=10):
    # load image and mask
    for i in range(n_images):
        # load the image and mask
        image = dataset.load_image(i)
        mask, _ = dataset.load_mask(i)
        # convert pixel values (e.g. center)
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)[0]
        # define subplot
        #pyplot.subplot(n_images, 2, i*2+1)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Actual')
        # plot masks
        for j in range(mask.shape[2]):
            pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
        # get the context for drawing boxes
        #pyplot.subplot(n_images, 2, i*2+2)
        # plot raw pixel data
        pyplot.savefig(image_prediction_folder+"/"+str(i)+"_actual.jpg")
        pyplot.show()

        pyplot.imshow(image)
        pyplot.title('Predicted')
        ax = pyplot.gca()
        # plot each box
        boxes_predicted = 0
        for box in yhat['rois']:
            # get coordinates
            y1, x1, y2, x2 = box
            # calculate width and height of the box
            width, height = x2 - x1, y2 - y1
            # create the shape
            rect = Rectangle((x1, y1), width, height, fill=False, color='red')
            # draw the box
            ax.add_patch(rect)
            boxes_predicted = boxes_predicted +1
    # show the figure
        pyplot.savefig(image_prediction_folder+"/"+str(i)+"_predicted.jpg")
        pyplot.show()
        print ("boxes_predicted: %d" % boxes_predicted)


In [ ]:

# load the train dataset
train_set = ObjectDataset()
train_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = ObjectDataset()
test_set.load_dataset('/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
#cfg_name = 'cfg20200301T1702'
#cfg_name = ''
cfg_name = 'cfg20200307T0901'
model_path = '/home/student/data_5000_project/programs/R-CNN/Mask_RCNN/opus_incertum_'+cfg_name+'/mask_rcnn_opus_incertum_cfg_0005.h5'
    
model.load_weights(model_path, by_name=True)
image_prediction_folder = '/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/predictions/'+cfg_name
if not os.path.exists(image_prediction_folder):
    os.mkdir(image_prediction_folder)

# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg, image_prediction_folder,1)
# plot predictions for test dataset
plot_actual_vs_predicted(test_set, model, cfg, image_prediction_folder,1)

In [ ]:
# display image with masks and bounding boxes
from os import listdir
import skimage
import random
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights

model_path = '/home/student/data_5000_project/programs/R-CNN/Mask_RCNN/opus_incertum_'+cfg_name+'/mask_rcnn_opus_incertum_cfg_0005.h5'
    
model.load_weights(model_path, by_name=True)

image_for_prediction_folder = '/storage/box/Pompeii in Pictures/Pompeii Ferebee/Pompeii/'

image_prediction_folder = '/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/predictions/'+cfg_name+"/unknown/"
if not os.path.exists(image_prediction_folder):
    os.mkdir(image_prediction_folder)

# Load a random image from the images folder
file_names = next(os.walk(image_for_prediction_folder))[2]
#print(file_names)
class_names = ["opus_incertum","opus_incertum","opus_incertum"]
for cn in range(1,20):
    
    image = skimage.io.imread(os.path.join(image_for_prediction_folder, random.choice(file_names)))
    # Run detection
    results = model.detect([image], verbose=1)
    # Visualize results
    r = results[0]
    image2= display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])


In [ ]:
# Run only once
image_for_prediction_folder = '/storage/box/Pompeii in Pictures/Pompeii Ferebee/Pompeii/'
file_names = next(os.walk(image_for_prediction_folder))[2]
#print(file_names)
test_file_names=[]
for cn in range(0,100):
    test_file_names.append(random.choice(file_names))

print(test_file_names)

import pickle
with open('rcnn_100_random_images_ferebee.pkl', 'wb') as f:
    #pickle.dump(test_file_names, f)

    
import pickle
with open('rcnn_100_random_images_ferebee.pkl', 'rb') as f:
    test_file_names = pickle.load(f)


In [ ]:
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_name, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+org_f_name+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>pompeiiinpictures.com</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<correct></correct>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close() 

In [ ]:
import pickle
import os
with open('rcnn_100_random_images_ferebee.pkl', 'rb') as f:
    test_file_names = pickle.load(f)

cfg_name = 'cfg20200307T0901'    
# display image with masks and bounding boxes
from os import listdir
import skimage
import random
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights

model_path = '/home/student/data_5000_project/programs/R-CNN/Mask_RCNN/opus_incertum_'+cfg_name+'/mask_rcnn_opus_incertum_cfg_0005.h5'
image_for_prediction_folder = '/storage/box/Pompeii in Pictures/Pompeii Ferebee/Pompeii/'    
model.load_weights(model_path, by_name=True)

image_prediction_folder = '/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/predictions/'+cfg_name+"/unknown/"
if not os.path.exists(image_prediction_folder):
    os.mkdir(image_prediction_folder)

#Store the results in XML    
class_names = ["opus_incertum","opus_incertum",]
for cn in range(1,5):
    
    image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
    #print("shape:", image.shape)
    # Run detection
    results = model.detect([image], verbose=1)
    # Visualize results
    r = results[0]
    #print(r['rois'], class_names, r['scores'])
    print(r['rois'][1],r['rois'][1][0],r['rois'][1][1],r['rois'][1][2],r['rois'][1][3])
    image2= display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])
    obj_f_num = "00000"+str(cn)
    obj_f_num = obj_f_num[-5:]
    refPts = []
    for rcn in range(0,len(r['rois'])):
        refPt = []
        refPtM = []
        refPtM.append(r['rois'][rcn][1])
        refPtM.append(r['rois'][rcn][0])
        refPt.append(refPtM)
        refPtM = []
        refPtM.append(r['rois'][rcn][3])
        refPtM.append(r['rois'][rcn][2])
        refPt.append(refPtM)
        refPts.append(refPt)
        print(refPts)
    write_annot(class_names[0], image_prediction_folder, obj_f_num, test_file_names[cn], image_for_prediction_folder,image.shape[0],image.shape[1],image.shape[2], refPts)


In [ ]:
display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            "oi", r['scores'])

In [ ]:
import cv2
for cn in range(1,2):
    
    
    image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
    
    obj_f_num = "00000"+str(cn)
    obj_f_num = obj_f_num[-5:]
    #print(fn)
    tree = ElementTree.parse(image_prediction_folder+obj_f_num+'.xml')
    print(image_prediction_folder+obj_f_num+'.xml')
    print(tree)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    #print(fn_image)
    box_num=0
    overlay = image.copy()
    output = image.copy()
    for box in root.findall('.//bndbox'):
        box_num=box_num+1
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        print(coors)
        cv2.rectangle(overlay, (xmin, ymin), (xmax, ymax),(0, 0, 255), -1) 
    alpha = .5
    cv2.addWeighted(overlay, alpha, output, 1-alpha,0, output)
    image = output
    scale_percent = 1024/image.shape[1] # percent of original size
    #print(scale_percent)
    width = int(image.shape[1] * scale_percent )
    height = int(image.shape[0] * scale_percent )
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized_original = resized.copy()
    

 # keep looping until the 'c' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", resized)
        cv2.moveWindow("image", 0,100)
        key = cv2.waitKey(1) & 0xFF

        # if the 'c' key is pressed, break from the loop
        if key == ord("0"):
            object_file_num_int_return = 0
            break
        if key == ord("w"):
            write_annot(object_type, object_annots_dir, object_file_num, original_image_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
            cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)
            object_file_num_int_return = object_file_num_int_return +1
            break
        if key == ord("-"):
            object_file_num_int_return = object_file_num_int_return-1
            break
        if key == ord("+"):
            object_file_num_int_return = object_file_num_int_return+1
            break
        if key == ord("1"):
            object_file_num_int_return = object_file_num_int_return+10
            break
        if key == ord("9"):
            object_file_num_int_return = object_file_num_int_return+90
            break            
        if key == ord("e"):
            object_file_num_int_return = -1
            break    

    #print(refPts)

    # close all open windows
    cv2.destroyAllWindows()
    image=None